<a href="https://colab.research.google.com/github/Nightmare125/Compiler/blob/main/Week_long_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

122589 Maina Richard Gichomo

ICS 4A

Task 1

In Python:

In [ ]:
def is_comment(line):
    # Check if a line starts with common comment indicators
    comment_indicators = ['//', '#', '/*', '*']
    for indicator in comment_indicators:
        if line.strip().startswith(indicator):
            return True
    return False

def identify_lines(lines):
    results = []
    for line in lines:
        if is_comment(line):
            results.append("Comment: " + line.strip())
        else:
            results.append("Code: " + line.strip())
    return results

# Example usage:
if __name__ == "__main__":
    code_lines = [
        "int x = 10; // Initialize x to 10",
        "print('Hello, world!')  # This is a comment",
        "/* This is a multi-line",
        "   comment */",
        "for i in range(5):",
        "    print(i) # Print the value of i",
    ]

    results = identify_lines(code_lines)
    for result in results:
        print(result)


i.) Logic Used:

*   The is_comment function determines whether a line begins with a common comment indicator like "//," "#," "/," or "." A line is regarded as a comment if it begins with one of these markers. If not, it qualifies as code.

*   The is_comment function is used by the identify_lines function to detect whether each line in a list of lines is a comment or a piece of code.    Each result in the list of results, which it returns, has a string indicating whether the line in question is code or a comment.

ii.) Lexical and Syntax Analysis:


*   This logic uses lexical analysis when we look for common comment indications at the beginning of each line. We search for specific character combinations that are frequently used in programming languages to represent comments.

*   This fundamental comment identification mechanism does not explicitly involve syntax analysis because we are not parsing or analyzing the code's structure. To make a straightforward determination, we are merely searching for evidence of frequent comments at the beginning of lines.


Task 2

Python Program to Validate Identifiers:

In [ ]:
import re

def is_valid_identifier(identifier):
    # Use a regular expression to validate the identifier
    # An identifier should start with a letter or underscore, followed by letters, digits, or underscores.
    # It should not be a keyword or a reserved word.
    keywords = ["if", "else", "while", "for", "return", "int", "float", "char", "bool"]
    if identifier in keywords:
        return False
    return re.match(r'^[a-zA-Z_][a-zA-Z0-9_]*$', identifier) is not None

# Example usage:
if __name__ == "__main__":
    identifier = input("Enter an identifier: ")
    if is_valid_identifier(identifier):
        print(f"'{identifier}' is a valid identifier.")
    else:
        print(f"'{identifier}' is not a valid identifier.")


To determine whether the provided identifier complies with the norms of most programming languages, this Python application employs a regular expression. It examines whether the identifier has letters, numbers, or underscores at the beginning, followed by a letter or underscore. The identification is also verified to ensure that it is neither a keyword or a reserved term.

Flex Specification File to Validate Identifiers:

In [ ]:
%{
#include <stdio.h>
%}

%%
[a-zA-Z_][a-zA-Z0-9_]* {
    printf("%s is a valid identifier.\n", yytext);
}
. {
    printf("%s is not a valid identifier.\n", yytext);
}
%%

int main() {
    yylex();
    return 0;
}


Task 3

Python program that constructs the LL(1) parsing table for the given CFG:

In [ ]:
class LL1Parser:
    def __init__(self, cfg):
        self.cfg = cfg
        self.non_terminals = set(cfg.keys())
        self.terminals = set()
        self.first_sets = {}
        self.follow_sets = {}
        self.parsing_table = {}

    def compute_first_sets(self):
        for non_terminal in self.non_terminals:
            self.first_sets[non_terminal] = set()
        while True:
            updated = False
            for non_terminal, productions in self.cfg.items():
                for production in productions:
                    for symbol in production:
                        if symbol in self.terminals:
                            if symbol not in self.first_sets[non_terminal]:
                                self.first_sets[non_terminal].add(symbol)
                                updated = True
                            break
                        elif symbol in self.non_terminals:
                            if not self.first_sets[symbol]:
                                break
                            if not self.first_sets[non_terminal].issuperset(self.first_sets[symbol]):
                                self.first_sets[non_terminal].update(self.first_sets[symbol])
                                updated = True
                        else:
                            self.terminals.add(symbol)
                            self.first_sets[non_terminal].add(symbol)
                            updated = True
            if not updated:
                break

    def compute_follow_sets(self, start_symbol):
        for non_terminal in self.non_terminals:
            self.follow_sets[non_terminal] = set()
        self.follow_sets[start_symbol].add('$')  # $ represents end of input
        while True:
            updated = False
            for non_terminal, productions in self.cfg.items():
                for production in productions:
                    for i, symbol in enumerate(production):
                        if symbol in self.non_terminals:
                            remaining_symbols = production[i + 1:]
                            first_of_remaining = self.first(remaining_symbols)
                            if '$' in first_of_remaining:
                                first_of_remaining.remove('$')
                                if not self.follow_sets[symbol].issuperset(self.follow_sets[non_terminal]):
                                    self.follow_sets[symbol].update(self.follow_sets[non_terminal])
                                    updated = True
                            self.follow_sets[symbol].update(first_of_remaining)
                            if all(x in first_of_remaining for x in remaining_symbols) and '$' in self.follow_sets[symbol]:
                                self.follow_sets[symbol].remove('$')
                                if not self.follow_sets[symbol].issuperset(self.follow_sets[non_terminal]):
                                    self.follow_sets[symbol].update(self.follow_sets[non_terminal])
                                    updated = True
            if not updated:
                break

    def first(self, symbols):
        result = set()
        for symbol in symbols:
            if symbol in self.terminals:
                result.add(symbol)
                break
            elif symbol in self.non_terminals:
                result.update(self.first_sets[symbol])
                if '$' not in self.first_sets[symbol]:
                    break
            else:
                result.add(symbol)
                break
        return result

    def construct_parsing_table(self):
        for non_terminal in self.non_terminals:
            for production in self.cfg[non_terminal]:
                first_of_production = self.first(production)
                for terminal in first_of_production:
                    if terminal != '$':
                        self.parsing_table[(non_terminal, terminal)] = production
                if '$' in first_of_production:
                    for terminal in self.follow_sets[non_terminal]:
                        self.parsing_table[(non_terminal, terminal)] = production

    def print_parsing_table(self):
        print("LL(1) Parsing Table:")
        print("{:<15} {:<15} {:<15}".format("Non-Terminal", "Terminal", "Production"))
        for (non_terminal, terminal), production in self.parsing_table.items():
            print("{:<15} {:<15} {:<15}".format(non_terminal, terminal, ' '.join(production)))

if __name__ == "__main__":
    # Define your CFG here
    cfg = {
        'S': ['aAB'],
        'A': ['b', '$'],
        'B': ['c', '$']
    }

    start_symbol = 'S'

    ll1_parser = LL1Parser(cfg)
    ll1_parser.compute_first_sets()
    ll1_parser.compute_follow_sets(start_symbol)
    ll1_parser.construct_parsing_table()
    ll1_parser.print_parsing_table()


In this program:

*   The LL1Parser class computes the FIRST and FOLLOW sets for each non-terminal and constructs the LL(1) parsing table.

*   The print_parsing_table method prints the LL(1) parsing table in a readable format.

